# Generate Map tiles from NYC taxi trip data

- 建立要繪製的 map tile 列表
  - 使用 Mercantile 套件，對設定的範圍自動產生所有的 Tile 列表。
- 統計 map tile 範圍內每個點的資料數量
  - 使用 datashader.Canvas.point 函數，對 map tile 範圍內進行統計。
  - 將統計結果使用 pickle 與 gzip 序列化成壓縮檔儲存（*.pkl.gz）。
  - 將每個點中的最大值儲存在 *.pkl.yaml 檔中方檢視。
  - 不儲存無資料（全為零）的統計結果以節省空間。
- 尋找 zoom 中的最大值
  - 統計 zoom 中的所有 tile 的最大值，並儲存在 _config.yaml 中。
- 依照統計資料，繪製每個 tile 圖片。
  - 讀取 *.pkl.gz 並還原為統計物件來進行繪圖。
  - 繪圖時使用 log 方法來進行內差計算，避免資料過於集中導致較少的點位看不出資料的問題。
  - 繪圖時將範圍設為 span=[0, zoom_agg__max]，避免不同 tile 做 interpolate 後得出的顏色範圍不統一的問題。
  - 儲存圖片到指定路徑。
- 使用 Folium 套件來呈現結果。


## 下載 NYC taxi trip data

In [1]:
import os
os.system("wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-05.csv")

0

## 讀取 dataset

In [2]:
import pandas as pd
import numpy as np

# 讀取 pickup_longitude 與 pickup_latitude 兩欄位
df = pd.read_csv("yellow_tripdata_2016-05.csv",
                 usecols=['pickup_longitude', "pickup_latitude"],
                 dtype={'pickup_longitude':np.float32,
                        "pickup_latitude":np.float32})

## 轉換 GPS 座標至 Web Mercator coordinate

因原始資料的座標使用的 GPS 資訊為球面座標，因此須先轉換為 Web mercator coordinate 才能使用在 map tiles 的座標系統中。我們使用 [pyproj](https://pypi.org/project/pyproj/) 來轉換座標

In [3]:
# 定義轉換函數
from pyproj import transform, Proj

# Set project function
source = Proj(init="epsg:4326") # WGS84
target = Proj(init="epsg:3857") # Web mercator 

# 轉換 Longitude 到 Web Mercator - X coordinate
def lngToX(lng):
    return np.float32(transform(source, target, lng, 0)[0])

# 轉換 Latitude 到 Web Mercator - Y coordinate
def latToY(lat):
    return np.float32(transform(source, target, 0, lat)[1])

df['pickup_x'] = df.pickup_longitude.apply(lngToX)
df['pickup_y'] = df.pickup_latitude.apply(latToY)


## 使用 Dask 平行處理 Dataframe

In [4]:
import dask.dataframe as dd
import multiprocessing as mp

dask_df = dd.from_pandas(df, npartitions=mp.cpu_count())
dask_df.persist()

,pickup_longitude,pickup_latitude,pickup_x,pickup_y
npartitions=2,,,,
0,float32,float32,float64,float64
5918427,...,...,...,...
11836852,...,...,...,...


## 建立所要產生的 Map Tile 列表

In [5]:
import mercantile

# New York City 周邊範圍
bound_nyc = (-74.029495, 40.697930, -73.946411, 40.817817) 

# 產生 zoom 範圍為 [0, 15] 的所有 tiles
tile_list = list(mercantile.tiles(*bound_nyc, zooms=list(range(0, 16))))


## 建立 Aggregation 資料並寫入檔案中

In [6]:
import datashader as ds
import os, pickle, gzip, yaml

from tqdm import tqdm

# 依照 Tile 位置建立 datashader.Canvas
def mapTilesCanvas(xtile, ytile, zoom, tile_size=(256, 256)):
    bounds = mercantile.xy_bounds(xtile, ytile, zoom)
    canvas = ds.Canvas(plot_width = tile_size[0],
                       plot_height = tile_size[1],
                       x_range = (bounds.left, bounds.right),
                       y_range = (bounds.bottom, bounds.top))
    return canvas

agg_root = './map/agg'

for tile in tqdm(tile_list):
    
    cvs = mapTilesCanvas(*tile)
    agg = cvs.points(dask_df, 'pickup_x', 'pickup_y')
    
    # 若 aggregation 為空則不產生檔案
    if (agg.values.max() < 1):
        continue
    
    # 依照與 map tile 相同的路徑儲存 aggregation 檔案
    # path = {root}/${zoom}/${x}/${y}
    agg_folder = os.path.join(agg_root, str(tile.z), str(tile.x))
    
    # 使用 Pickle 序列化 Aggregation 並儲存成  gzip 
    agg_file = os.path.join(agg_folder, str(tile.y) + ".pkl.gz")
    os.makedirs(os.path.dirname(agg_file), exist_ok=True)
    with gzip.open(agg_file, mode='wb') as file:
        pickle.dump(agg, file)
    
    # 建立 Aggregation 檔案的 Yaml 檔
    agg_yaml_file = os.path.join(agg_folder, str(tile.y) + ".pkl.yaml")
    os.makedirs(os.path.dirname(agg_yaml_file), exist_ok=True)
    with open(agg_yaml_file, mode='w') as file:
        yaml_obj = {"agg_file": os.path.basename(agg_file),
                    "max_count": int(agg.values.max()),
                    "tile": {"z": tile.z, "x": tile.x, "y":tile.y }
                   }
        yaml.dump(yaml_obj, file, default_flow_style=False)
    

100%|██████████| 212/212 [00:24<00:00,  8.68it/s]


## 建立每個 zoom 中的點位最大值資料

In [7]:
# 取得儲存 Yaml 檔案中，最大點數的資訊
def getYamlMaxCount(yaml_file):
    with open(yaml_file, 'r') as file:
        try:
            return yaml.load(file)['max_count']
        except BaseException:
            return 0
    return 0

# 計算 zoom 中的最大點數資料
def getZoomMaxCount(base_root):
    max_count = 0;
    for root, dirs, files in os.walk(base_root):
        for file in files:
            exts = file.split(os.extsep)

            if len(exts) < 3:
                continue
                
            if exts[-2] != 'pkl':
                continue

            if not (exts[-1] == "yaml" or exts[-1] == "yml"):
                continue
                
            mc = getYamlMaxCount(os.path.join(root, file))
            if max_count < mc:
                max_count = mc

    return max_count


zoom_list = os.listdir(agg_root)

for zoom in zoom_list:
    max_count = getZoomMaxCount(os.path.join(agg_root, zoom))
    zoom_yaml = os.path.join(agg_root, zoom, '_config.yaml')
    
    with open(zoom_yaml, 'w') as file:
        yaml_obj = {'zoom_max_count': max_count}
        yaml.dump(yaml_obj, file, default_flow_style=False)

## 產生每個 Aggregation 檔案對應的 Tile 影像

In [8]:

# 由 path 得出 aggregation file 對應的 map tile 路徑
def getMapTileCoord(agg_path):
    sep = agg_path.split(os.sep)
    if len(sep) < 3:
        raise ValueError("agg_path can not convert to tile path")
    
    z = sep[-3]
    x = sep[-2]
    y = sep[-1].split('.')[0]
        
    return (z, x, y)

# 從 _config.yaml 檔建立
def getMaxCountDict(agg_root):
    max_dict = {}
    
    for folder in os.listdir(agg_root):
        zoom_conf_f = os.path.join(agg_root, folder, '_config.yaml')
        with open(zoom_conf_f, 'r') as f:
            try:
                obj = yaml.load(f)
                max_dict[folder] = int(obj['zoom_max_count'])
            except:
                print('Get max count in zoom ', folder, ': Error')
            
    return max_dict


In [9]:
import datashader.transfer_functions as tf
from matplotlib.cm import hot

tile_root = 'map/tile'

max_dict = getMaxCountDict(agg_root)

for root, dirs, files in os.walk(agg_root):
    for file in files:
        
        sep = file.split(".")
        if len(sep) < 3:
            continue
            
        if sep[-1] != 'gz' or sep[-2] != 'pkl':
            continue
            
        agg_path = os.path.join(root, file)
        
        with gzip.open(agg_path, 'rb') as f:
            agg = pickle.load(f)
            
            zoom, xtile, ytile = getMapTileCoord(agg_path)
            
            if zoom in max_dict.keys():
                img = tf.shade(agg, cmap=hot, how='log', span=[0, max_dict[zoom]])
            else:
                img = tf.shade(agg, cmap=hot, how='log')
            
            tile_path = os.path.join(tile_root, zoom, xtile, ytile + '.png')
            os.makedirs(os.path.dirname(tile_path), exist_ok=True)
            with open(tile_path, mode='wb') as out:
                out.write(img.to_bytesio(format='png').read())


/home/peter_li/miniconda3/lib/python3.6/site-packages/matplotlib/colors.py:504: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


## 使用 Folium 顯示圖層

In [10]:
import folium

# 使用 Carto Dark 建立底圖
fmap = folium.Map(location=[40.772562, -73.974039],
                  tiles='https://cartodb-basemaps-{s}.global.ssl.fastly.net/dark_all/{z}/{x}/{y}.png',
                  zoom_start=12,
                  attr='Carto Dark')
, 
# 加入放在 GitHub 存放的 map tiles 位置
fmap.add_tile_layer(tiles='https://raw.githubusercontent.com/yeshuanova/nyc_taxi_trip_map/master/map/tile/{z}/{x}/{y}.png',
                    attr='NYC taxi pickup Heatmap')

# 儲存地圖為 Html 檔，可直接用瀏覽器打開檢視地圖資訊
fmap.save('map.html')

# 直接在 ipython 中顯示地圖
fmap
